In [1]:
#loading the packages#

#using Pkg
#Pkg.add("DataFrames")
#Pkg.add("Random")
#Pkg.add("RollingFunctions")
#Pkg.add("XLSX")
#Pkg.add("Distributions") 

using DataFrames
using Random
using RollingFunctions
using XLSX
using Distributions

In [2]:
# importing the dataframe #

#this one has all the values
s7_data_combined = XLSX.readxlsx("C:/Users/15167/OneDrive/Documents/ISB/AML-DT-BNM/raw_data/s7_data_combined.xlsx")
sh = s7_data_combined["Sheet1"]

608×2 XLSX.Worksheet: ["Sheet1"](A1:B608) 

In [3]:
# setting the variables and constants #

n_obs = 20000  # number of steps
n_constant_nodes = ((n_obs) - 5000)  # for the constant nodes (15000 steps)
p = 78  # number of variables
chance_to_flip = 0.01

#78 time-series of noise for 78 variables  #integers not floats
zeta = rand(Uniform(0,1), p, n_obs)

#creates an "empty" time series  #integers not floats
empty_series = zeros(Int, p, n_obs)

# Initial value of the variables were randomly chosen to be 0.0 or 1.0
empty_series[:, 1] = rand(0:1, p)

# Initial value of the variables were randomly chosen to be 0.0 or 1.0 
empty_series[:, n_constant_nodes] = rand(0:1, p)

# Saving the empty time series as the time series (so it can be regenerated later)
time_series = empty_series

# Saving a network_score matrix 
network_scores = zeros(n_obs, 5)

#example mutation_profile to see if function works
mutation_profile = sh["B2"]

#mutation_profile = sh["B1:B20"] #getting all the mutation profiles ONLY
#time_series[1,:] #gives you a row (this would be an individual node in our case) with n.obs

"WT1,FLT3,DNMT3A,DNMT3A"

In [7]:
# score_personal function #

#creating the function for personal score
function score_personal(mutation_profile)
    # Separate the values by commas
    mutation_profile = split(mutation_profile, ',')

    # Remove repeated values
    mutation_profile = unique(mutation_profile)
    
    # Join back together
    join(mutation_profile, ",")

    println(mutation_profile)
    println("preparing to create time series")
    
    for col in 2:n_obs
        #simulate each time step based on the previous time step and Boolean functions
        time_series[1,col]  = time_series[1,col-1] #FLT3 = FLT3
        time_series[2,col]  = time_series[1,col-1] #AKT= FLT3
        time_series[3,col]  = abs((time_series[1,col-1])-1) #CEBPA=!FLT3
        time_series[4,col]  = time_series[4,col-1] #DNMT3A= DNMT3A
        time_series[5,col]  = abs((time_series[2,col-1])-1) #GSK3B= !AKT
        time_series[6,col]  = time_series[6,col-1] #NPM1= NPM1
        time_series[7,col]  = time_series[6,col-1] #ARF= NPM1
        time_series[8,col]  = abs((time_series[6,col-1])-1) #HOXA9= !NPM1
        time_series[9,col]  = time_series[6,col-1] #FBXW7= NPM1
        time_series[10,col]  = time_series[1,col-1] #ERK= FLT3
        time_series[11,col]  = time_series[6,col-1] #CDKN2A= NPM1
        time_series[12,col]  = time_series[1,col-1] #STAT5A= FLT3
        time_series[13,col]  = abs((time_series[3,col-1])-1) #SOX4= !CEBPA
        time_series[14,col] = abs((time_series[4,col-1])-1) & abs((time_series[5,col-1])-1) #CCND1= !(DNMT3A | GSK3B)
        time_series[15,col] = abs((time_series[4,col-1])-1) & time_series[8,col-1] #MEIS1=!(DNMT3A & !(HOXA9)
        time_series[16,col] = abs((time_series[5,col-1]) -1) & abs((time_series[9,col-1])-1) & time_series[10,col-1]  #MYC= !(GSK3B & FBXW7) & ERK
        time_series[17,col]  = abs((time_series[10,col-1])-1) #ETV6=!ERK
        time_series[18,col]  = abs((time_series[7,col-1]) -1) #TP53= ARF
        time_series[19,col]  = time_series[10,col-1] & abs((time_series[18,col-1])-1) #BCL2= ERK & TP53
        
        #apply random noise
        noise = rand(p) .< chance_to_flip
        time_series[:, col] .= xor.(time_series[:, col], noise)
        
    end
    
    #knocking in/out genes
    for col in 1:n_obs
        if any(x -> occursin.("FLT3", x), mutation_profile)
            time_series[1,col] = 1
        end 
        
        if any(x -> occursin.("AKT", x), mutation_profile)
            time_series[2,col] = 1
        end 
        
        if any(x -> occursin.("CEBPA", x), mutation_profile)
            time_series[3,col] = 1
        end 
        
        if any(x -> occursin.("DNMT3A", x), mutation_profile)
            time_series[4,col] = 0
        end 
        
        if any(x -> occursin.("GSK3B", x), mutation_profile)
            time_series[5,col] = 1
        end
        
        if any(x -> occursin.("NPM1", x), mutation_profile)
            time_series[6,col] = 1
        end
        
        if any(x -> occursin.("ARF", x), mutation_profile)
            time_series[7,col] = 1
        end
        
        if any(x -> occursin.("HOXA9", x), mutation_profile)
            time_series[8,col] = 1
        end
        
        if any(x -> occursin.("FBXW7", x), mutation_profile)
            time_series[9,col] = 1
        end
        
        if any(x -> occursin.("ERK", x), mutation_profile)
            time_series[10,col] = 1
        end 
        
        if any(x -> occursin.("CDKN2A", x), mutation_profile)
            time_series[11,col] = 1
        end 
        
        if any(x -> occursin.("STAT5A", x), mutation_profile)
            time_series[12,col] = 1
        end 
        
        if any(x -> occursin.("SOX4", x), mutation_profile)
            time_series[13,col] = 1
        end 
        
        if any(x -> occursin.("CCND1", x), mutation_profile)
            time_series[14,col] = 1
        end
        
        if any(x -> occursin.("MEIS1", x), mutation_profile)
            time_series[15,col] = 1
        end
        
        if any(x -> occursin.("MYC", x), mutation_profile)
            time_series[16,col] = 1
        end
        
        if any(x -> occursin.("ETV6", x), mutation_profile)
            time_series[17,col] = 1
        end
        
        if any(x -> occursin.("TP53", x), mutation_profile)
            time_series[18,col] = 1
        end
        
        if any(x -> occursin.("BCL2", x), mutation_profile)
            time_series[19,col] = 1
        end
    end
    
    println("time series is complete")
    
	#computing the scores
	for col in 1:n_obs
		network_scores[col,1] = ((-1 * time_series[67,col]) + (-1 * time_series[69,col]) + (-1 * time_series[70,col]) + time_series[29,col] + time_series[34,col] + time_series[61,col] + time_series[57,col] + time_series[23,col] + time_series[72,col] + time_series[65,col] + time_series[71,col])
		network_scores[col,2]= ((-1 * time_series[30,col]) + ( -1 * time_series[45,col]) + time_series[32,col] + time_series[50,col] + time_series[24,col] + time_series[56,col] + time_series[40,col])
		network_scores[col,3] = ((-1 * time_series[78,col]) + (-1 * time_series[33,col]) + time_series[39,col] + time_series[77,col] + time_series[28,col])
		network_scores[col,4] = (network_scores[col,1]) - ((network_scores[col,2]) + (network_scores[col,3]))
    end 
    
    #computing the average scores from rolling mean
	#rolling window of size n_obs/1000
	window_size = div(n_obs, 2000)
	
	#calculate the rolling average for each variable using the RollingFunctions package
	rolling_avg = rollmean(network_scores[:,4], window_size)
	
	#Compute the final_score
    final_score = mean(rolling_avg[(length(rolling_avg)-2000):length(rolling_avg)], dims=1)
	final_score = final_score[1]
	final_score = round(final_score, sigdigits=3)
    
    proliferation = mean(network_scores[15000:20000,1])
    differentiation = mean(network_scores[15000:20000,2])
    apoptosis = mean(network_scores[15000:20000,3])
    final_score = final_score
    
    score = [proliferation,differentiation,apoptosis,final_score]
    
    return score

end

score_personal (generic function with 1 method)

In [8]:
#mutation_profile = sh["B1:B20"] 

In [9]:
# Using the function #
mutation_profile = sh["B1:B608"] #getting all the mutation profiles ONLY
#mutation_profile = sh["B1:B5"] #getting all the mutation profiles ONLY

# Saving a prolif, diff, apoptosis score matrix
scores = zeros(608,4)

for i in 1:(length(mutation_profile))
    
    #using the function
    result = score_personal(mutation_profile[i]) 
    println(result)
    
    #saving the scores
    scores[i,1]=result[1] #proliferation
    scores[i,2]=result[2] #differentiation
    scores[i,3]=result[3] #apoptosis
    scores[i,4]=result[4] #final network score

    time_series = empty_series
end

SubString{String}["TET2", "NPM1"]
preparing to create time series
time series is complete
[0.11137772445510898, 0.05298940211957608, 0.018196360727854428, 0.0303]
SubString{String}["WT1", "FLT3", "DNMT3A"]
preparing to create time series
time series is complete
[0.15656868626274745, 0.08078384323135374, 0.02959408118376325, 0.0563]
SubString{String}["ETV6", "FLT3", "SRSF2", "SETBP1"]
preparing to create time series
time series is complete
[0.20415916816636673, 0.11237752449510098, 0.03719256148770246, 0.088]
SubString{String}["FLT3", "CBLB", "TET2", "NPM1"]
preparing to create time series
time series is complete
[0.2615476904619076, 0.14117176564687062, 0.05138972205558888, 0.109]
SubString{String}["WT1", "FLT3", "DNMT3A"]
preparing to create time series
time series is complete
[0.3001399720055989, 0.16616676664667065, 0.0609878024395121, 0.115]
SubString{String}["FLT3", "HOXA1"]
preparing to create time series
time series is complete
[0.34493101379724056, 0.19096180763847231, 0.073185

time series is complete
[1.9888022395520897, 1.2181563687262547, 0.39752049590081984, 0.308]
SubString{String}["SCNM1", "TGFB2", "MAX", "CACNA1G", "PARP14", "ZNF711"]
preparing to create time series
time series is complete
[1.9886022795440912, 1.219756048790242, 0.3997200559888022, 0.3]
SubString{String}["DIRAS3", "F5", "HEATR1", "SLC22A24", "SLAIN1", "SYNGR3", "SRSF2", "SIRT2", "DUXA", "C2orf74", "ABHD16B", "RUNX1", "FOXP1", "PDZRN3", "GPER1", "SLC4A2", "KIAA2026", "BCOR", "PHF6", "BRCC3"]
preparing to create time series
time series is complete
[2.002399520095981, 1.2263547290541892, 0.40091981603679266, 0.308]
SubString{String}["KRT10", "TET2", "CNPY3"]
preparing to create time series
time series is complete
[2.0067986402719455, 1.2305538892221555, 0.3977204559088182, 0.318]
SubString{String}["UBE2J2", "HTR1D", "COL24A1", "THBS3", "PGBD5", "CAPN5", "FLT3", "SERPINA11", "PTPRS", "PPP1R13L", "KLK13", "SF3B1", "SULF2", "RUNX1", "UGT2B10", "USP9X"]
preparing to create time series
time se

time series is complete
[2.2319536092781442, 1.3589282143571286, 0.4377124575084983, 0.441]
SubString{String}["NRAS", "PLEKHG6", "SLC25A41", "MEGF8", "ZNF638", "NCKAP1", "ASXL1", "NDNF", "XKR4"]
preparing to create time series
time series is complete
[2.2299540091981602, 1.368126374725055, 0.4405118976204759, 0.422]
SubString{String}["RPS6KA1", "RIT1", "GCSAML", "PLCB3", "DPY19L2", "MRC2", "U2AF1", "BMP2K", "NPR3", "MYL10", "JAK2"]
preparing to create time series
time series is complete
[2.23875224955009, 1.3727254549090182, 0.4453109378124375, 0.423]
SubString{String}["KMT2A", "APOBEC1", "TSPAN8", "FLT3", "RNF113B", "SMOC1", "INF2", "IDH2", "GPATCH8", "ZNF560", "SBF1", "NUP210", "NPM1", "ZNF292", "OFD1"]
preparing to create time series
time series is complete
[2.2505498900219956, 1.3795240951809637, 0.4465106978604279, 0.424]
SubString{String}["CRY2", "PTPRB", "UNG", "FLT3", "TMC3", "ZNF587", "SPRED2", "FAM194A", "EGR1", "NPM1", "COL28A1"]
preparing to create time series
time series i

time series is complete
[2.372725454909018, 1.4317136572685463, 0.48530293941211755, 0.367]
SubString{String}["TP53", "MIEF2", "SRSF2", "ASXL1"]
preparing to create time series
time series is complete
[2.3745250949810037, 1.435112977404519, 0.48210357928414316, 0.38]
SubString{String}["DDX41", "LMOD2"]
preparing to create time series
time series is complete
[2.3705258948210357, 1.437112577484503, 0.4809038192361528, 0.374]
SubString{String}["ATAD3B", "ARHGEF10L", "TBC1D10C", "KRAS", "TLN2", "ACAN", "CACNA1G", "MYO5B", "NUMBL", "ARHGAP40", "NBEAL2", "KCTD20", "COL19A1", "IKZF1", "NOBOX", "CACNA1B", "ITGB1BP2"]
preparing to create time series
time series is complete
[2.3685262947410517, 1.4389122175564888, 0.48170365926814634, 0.365]
SubString{String}["SLC22A10", "FNDC3A", "DNMT3A", "PLXNB1", "MAP1B", "TLE1"]
preparing to create time series
time series is complete
[2.367726454709058, 1.4413117376524696, 0.47590481903619275, 0.384]
SubString{String}["TIPRL", "PRKCDBP", "ADRBK1", "EVL", "S

preparing to create time series
time series is complete
[2.4311137772445512, 1.4693061387722455, 0.5030993801239753, 0.405]
SubString{String}["RIT1", "NOC3L", "AP2A2", "WT1", "NAT10", "HMGB1", "MARK3", "TRPM1", "KRT10", "CCDC68", "SLC8A2", "TRPM4", "NLRP13", "AC012313.1", "HOXD13", "PKDREJ", "C6orf58", "TAS2R38", "FAM135B"]
preparing to create time series
time series is complete
[2.431513697260548, 1.4717056588682265, 0.5040991801639672, 0.408]
SubString{String}["MRPL24", "ZRANB1", "MYH7B"]
preparing to create time series
time series is complete
[2.4347130573885223, 1.4751049790041992, 0.5046990601879624, 0.423]
SubString{String}["NRAS", "GJA8", "RIN1", "PZP", "TP53", "ALKBH5", "POLI", "ZNF563", "ANKRD60", "RASA2", "C4orf21", "SLC34A1", "PHYKPL", "PLXNA4", "HUWE1", "L1CAM"]
preparing to create time series
time series is complete
[2.430113977204559, 1.475504899020196, 0.5028994201159768, 0.421]
SubString{String}["ZNF195", "WT1", "WNK1", "MON2", "FLT3", "PPL", "AHRR", "MRS2"]
preparing t

...

time series is complete
[2.5274945010997802, 1.4817036592681463, 0.5036992601479704, 0.544]
SubString{String}["ATAD3A", "NRAS", "TNKS1BP1", "GRIN2B", "CRY1", "ADPRHL1", "PGLYRP2", "MYT1", "GPR111", "CHD7"]
preparing to create time series
time series is complete
[2.527894421115777, 1.4781043791241752, 0.5036992601479704, 0.546]
SubString{String}["RHNO1", "AC024580.1", "ASXL1", "CCKAR", "TET2", "BRAF"]
preparing to create time series
time series is complete
[2.529894021195761, 1.4843031393721255, 0.4997000599880024, 0.555]
SubString{String}["ARID1A", "EPHA10", "C1orf101", "IGHV2-5", "TPSD1", "C17orf97", "RSPH6A", "TUBB1", "KIT", "ANKS6", "MXRA5"]
preparing to create time series
time series is complete
[2.5236952609478105, 1.486502699460108, 0.499500099980004, 0.546]
SubString{String}["WDR47", "CBL", "IDH2", "PLD3", "SOGA1", "PIK3R4", "RFC4", "NPM1"]
preparing to create time series
time series is complete
[2.5266946610677863, 1.4907018596280743, 0.4929014197160568, 0.546]
SubString{String

time series is complete
[2.5386922615476903, 1.4993001399720056, 0.47470505898820237, 0.535]
SubString{String}["FLT3"]
preparing to create time series
time series is complete
[2.541891621675665, 1.4991001799640071, 0.47530493901219756, 0.528]
SubString{String}["TTC17", "FLT3", "AURKB", "FCHO1", "DNMT3A", "C2ORF15", "A4GALT", "ROBO1", "SMR3B", "HTR4", "ID4", "ZFHX4", "FAM27D1", "MED12"]
preparing to create time series
time series is complete
[2.5412917416516696, 1.4907018596280743, 0.47390521895620874, 0.531]
SubString{String}["NRAS", "DNA2", "GOT1", "WT1", "FLT3", "NPC1", "URB1", "NPM1", "ZNF777", "AMER1"]
preparing to create time series
time series is complete
[2.540891821635673, 1.4899020195960808, 0.47570485902819437, 0.534]
SubString{String}["SLC16A4", "GNAS", "RUNX1"]
preparing to create time series
time series is complete
[2.541891621675665, 1.4937012597480503, 0.47630473905218956, 0.531]
SubString{String}["POU3F3", "GCM1"]
preparing to create time series
time series is complete


time series is complete
[2.524495100979804, 1.5020995800839831, 0.5050989802039592, 0.535]
SubString{String}["NPAP1", "TP53", "KIT"]
preparing to create time series
time series is complete
[2.5190961807638472, 1.5022995400919816, 0.5046990601879624, 0.538]
SubString{String}["NRAS", "ASXL1"]
preparing to create time series
time series is complete
[2.520495900819836, 1.498500299940012, 0.4979004199160168, 0.552]
SubString{String}["NPHP4", "ARID1A", "PDS5B", "ANKRD63", "SRSF2", "ASXL1", "TBRG4", "AGTPBP1", "BCOR"]
preparing to create time series
time series is complete
[2.5258948210357928, 1.4949010197960408, 0.49610077984403117, 0.559]
SubString{String}["FAM131C", "CCDC181", "PRKAG1", "RASGRF1", "CEBPA", "VAX2", "IDH1", "GRID2", "LRP12", "KCNT1", "ZNF711"]
preparing to create time series
time series is complete
[2.524495100979804, 1.4971005798840231, 0.501499700059988, 0.554]
SubString{String}["DHRS2", "TRIM47", "TRAPPC8", "USP20"]
preparing to create time series
time series is complete


preparing to create time series
time series is complete
[2.494501099780044, 1.5442911417716456, 0.50749850029994, 0.451]
SubString{String}["PGR", "IDH2", "KCNG4", "KLHL36", "SRSF2", "FAM210A", "MED16", "KLF7", "ITSN1", "NKX6-1", "SLC6A14", "DCAF12L1"]
preparing to create time series
time series is complete
[2.4919016196760646, 1.5440911817636473, 0.5086982603479304, 0.445]
SubString{String}["WDR63", "HSPA6", "ZNF236", "DNMT3A", "ASB1", "SDHA", "DENND1A"]
preparing to create time series
time series is complete
[2.49750049990002, 1.5350929814037193, 0.5068986202759448, 0.466]
SubString{String}["CSMD2", "ATG2A", "FLT3", "SYNM", "RSL1D1", "NMU", "C5orf42", "NPM1", "TSPYL4", "LRRCC1", "BARHL1", "FAM155B"]
preparing to create time series
time series is complete
[2.4957008598280344, 1.5386922615476906, 0.5066986602679464, 0.466]
SubString{String}["NRAS"]
preparing to create time series
time series is complete
[2.496900619876025, 1.5396920615876826, 0.499500099980004, 0.475]
SubString{String}[

time series is complete
[2.5178964207158567, 1.5336932613477305, 0.5048990201959608, 0.55]
SubString{String}["NRAS", "WT1", "NPM1"]
preparing to create time series
time series is complete
[2.524495100979804, 1.5368926214757048, 0.50249950009998, 0.553]
SubString{String}["KRT14", "ASXL1"]
preparing to create time series
time series is complete
[2.524095180963807, 1.5372925414917016, 0.504499100179964, 0.561]
SubString{String}["NRAS", "C1QL3", "SMC3", "TPTE2", "C17orf62", "GTF2F1", "ADAMTS10", "DNMT3A", "IDH1", "CCDC157", "ANKRD50", "NPM1", "TRDN", "TRRAP"]
preparing to create time series
time series is complete
[2.522895420915817, 1.5298940211957608, 0.501499700059988, 0.574]
SubString{String}["PAPPA2", "KRTAP2-4", "SYT4", "CLTCL1", "ABCF3", "ME1", "AOC1", "PNMA3"]
preparing to create time series
time series is complete
[2.5236952609478105, 1.531493701259748, 0.506498700259948, 0.582]
SubString{String}["WT1", "KRR1", "NF1", "DNMT3A", "TTLL9", "NPM1", "CUX1"]
preparing to create time ser

In [10]:
column_names = [:Proliferation, :Differentation, :Apoptosis, :Final]
scores_df = DataFrame(scores, column_names)

Row,Proliferation,Differentation,Apoptosis,Final
,Float64,Float64,Float64,Float64
1,0.111378,0.0529894,0.0181964,0.0303
2,0.156569,0.0807838,0.0295941,0.0563
3,0.204159,0.112378,0.0371926,0.088
4,0.261548,0.141172,0.0513897,0.109
5,0.30014,0.166167,0.0609878,0.115
6,0.344931,0.190962,0.0731854,0.124
7,0.387922,0.217756,0.0795841,0.144
8,0.429514,0.240152,0.0923815,0.157
9,0.461908,0.263347,0.0961808,0.16


In [11]:
#saving the score matrix as an xlsx file for R 
xlsx_file_path= "C:/Users/15167/OneDrive/Documents/ISB/AML-DT-BNM/raw_data/julia_dnmt3a.xlsx"

XLSX.writetable(xlsx_file_path, collect(DataFrames.eachcol(scores_df)), DataFrames.names(scores_df))